Quick Drawing is a kaggle challenge for classifying the sketch pictures. The total number of category is 340 and 
there are 50M pictures in the dataset. Each catogory has 25,000 pictures. The difficulty is the drawing might be 
incomplete or the drawing does not match the label. We are trying to develop a CNN to solve the issue.   

According to the challenge, there are 4 kinds of dataset. 

    - Raw data (.ndjson)
    - Simplified drawings (.ndjson)
    - Binary data (.bin)
    - Bitmap data (.npy)
Raw data and Simplified drawings are provided in a vector information of strokes. Bitmap data is rendered into 
28 * 28 grayscale bitmap in the numpy format.  

Maybe we could consider the raw data to improve the performance since it inlcudes the sequence information.


In [1]:
import torch
import time
import torch
import os
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
from torch.utils import data
import QDdata as QD
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
import resnet as RN
from torchvision import transforms
import csv

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 300

print('using device:', device)


using device: cuda


# Data Augmentation

In [3]:
trans = transforms.Compose([transforms.RandomHorizontalFlip(0.5), transforms.ToTensor()])

# Center Feature Per Class

In [4]:
alpha = 0.01
cf_class = torch.from_numpy(np.load("center_feature_ssn.npy"))
cf_class = cf_class.to(device=device, dtype=torch.float32)

In [5]:
args = {"learning_rate" : 3e-3, "model" : "cnn"}

In [6]:
# The following code comes from https://github.com/pytorch/examples/blob/master/imagenet/main.py
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def adjust_learning_rate(optimizer, epoch, args):
    """Sets the learning rate to the initial LR decayed by 0.5 every epoch"""
    lr = args['learning_rate'] * (0.5 ** epoch )
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [7]:
class SketchRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout = 0.1, n_layers=1):
        super(SketchRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.conv1d_1 = nn.Conv1d(input_size, 48, 5)
        self.dropout_1 = nn.Dropout(0.1)
        self.conv1d_2 = nn.Conv1d(48, 64, 5)
        self.dropout_2 = nn.Dropout(0.1)
        self.conv1d_3 = nn.Conv1d(64, 96, 3)
        self.dropout_3 = nn.Dropout(0.1)
        self.lstm_1 = nn.LSTM(96,hidden_size, n_layers, dropout,batch_first=True,bidirectional=True)
        self.fc_mu1 = nn.Linear(hidden_size*186*2, output_size)
        #self.fc_mu2 = nn.Linear(128, output_size)

    def forward(self, inputs, hidden):
        inputs = inputs.transpose(1, 2)
        
        output = self.conv1d_1(inputs)
        output = self.dropout_1(output)
        output = self.conv1d_2(output)
        output = self.dropout_2(output)
        output = self.conv1d_3(output)
        output = self.dropout_3(output)
        output = output.transpose(1, 2)
        
        output, (hidden,x) = self.lstm_1(output, hidden)
        
        output = output.contiguous()
        output = output.view(output.size(0),-1)
        output_lstm = self.fc_mu1(output)
        #output = self.fc_mu2(output_lstm)
        output = F.log_softmax(output_lstm, dim=1)
        return output, output_lstm 

# Resnet50 96X96

In [8]:
###### The following codes comes from cs231N.
from average_precision import mapk
def mean_ap_torch(y_val,y_pred_score,k):
    y_pred = [torch.sort(c,descending=True)[::-1][1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def mean_ap_3(y_val,y_pred_score,k):
    y_pred = [np.argsort(c)[::-1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def check_accuracy(loader, model):
    
    model.eval()  # set model to evaluation mode
    
    meanAP = []
    meanAP3 = []
    with torch.no_grad():
        for i ,(x,stroke, y) in enumerate(loader):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            y_pred,_ = model(x)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
            if i > 100:
                break
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        

        if (m3 > QDtrain["mAP3"] + 1) and (m3 > 20):
            if os.path.exists(QDtrain["file"]):
                os.remove(QDtrain["file"])
            QDtrain["file"] = str("model" + QDtrain["prefix"] + str(m3))
            torch.save(model.state_dict(),QDtrain["file"])
            QDtrain["mAP3"] = m3
            QDtrain["mAP"] = m1
        print('Got correct (%.2f,%.2f)' % (m1,m3 ))
        
        
def train(model, optimizer, epochs,args):
    """
    Train a model on quickdrawing.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: A Python integer giving the number of epochs to train for
    - args: argumetns for learning rate
    
    Returns: Nothing, but prints model accuracies during training.
    """

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loader_val = QD.QDloadStrokeData(val = True)
    loader_val = DataLoader(dataset=loader_val, batch_size=256,shuffle=False)
    
    # We try to aggregate the several batches together 
    # so that we could have a big batchsize to fill in GPU.
    # real_batch size = aggregated_batches * batch_size
    aggregated_batches = 1
    
    for e in range(epochs):

        train_map1 = AverageMeter()
        train_map3 = AverageMeter()
        
        #Learning rate decay
        adjust_learning_rate(optimizer, e, args)
        
        # We split the whole train dataset into 100 segments.
        for i in range(20):
            t1 = time.time()
            total_loss = 0
            train_dataset = QD.QDloadStrokeData(no=i,val = False,transforms = trans)          
            train_loader = DataLoader(dataset=train_dataset, batch_size=256,shuffle=False)
            for t, (x,stroke, y) in enumerate(train_loader):
                model.train()  
                x = x.to(device=device, dtype=dtype)  
                y = y.to(device=device, dtype=torch.long)

                #add the center feature returned from resnet 
                scores,cf_pred = model(x)
                
                #Caculate entropy loss
                entropy_loss = F.cross_entropy(scores, y)
                
                #Caculate the center loss 
                center_loss = F.mse_loss(cf_pred,cf_class[y])
                
                loss = entropy_loss + alpha * center_loss
                
                total_loss += loss
        
                                  
                if t % aggregated_batches == 0:                    
                    avg_loss = total_loss / aggregated_batches
                    
                    #Calculate train accuracy
                    y_pred = scores.data.cpu().numpy()
                    y_val = y.reshape(-1,1)

                    mAP = 100 * mean_ap_3(y_val,y_pred,1)
                    mAP3 = 100 * mean_ap_3(y_val,y_pred,3)

                    #train_map1.update(mAP,x.size(0))
                    #train_map3.update(mAP3,x.size(0))
                    
                    optimizer.zero_grad()
                    avg_loss.backward()
                    optimizer.step()
                    total_loss = 0

                    t2 = time.time()

                    if t % (print_every) == 0:
                        print('Epoch %d,Iteration %d,loss = %.4f,time %.4f,train accuracy(%.2f,%.2f)' % 
                              (e,t, avg_loss.item(),t2-t1,mAP,mAP3))
                        check_accuracy(loader_val, model)
                        t1 = time.time()
                        

In [9]:
cnn_model = RN.resnet50(num_classes=40)
optimizer = optim.Adam(cnn_model.parameters(),lr = args['learning_rate'])
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"resnet50"}

In [ ]:
train(cnn_model,optimizer, epochs=5,args=args)

1. batch size = 64. count =  340 * 10240.
   Iteration 29400, loss = 0.5610
   Got 64 correct (80.22,85.62)
2. batch size =128 count = 340 *10240 * 2
    Iteration 37800, loss = 0.9535
    Got 128 correct (77.65,83.64)
3. batch size =128 count = 340 *10240 * 2 
    train on 28 * 28 drawing transfered from stroke
    Iteration 45900, loss = 0.9149
    Got correct (57.55,65.98)

## RNN Model

In [ ]:
# The following codes comes from cs231N.
from average_precision import mapk
def mean_ap_torch(y_val,y_pred_score,k):
    y_pred = [torch.sort(c,descending=True)[::-1][1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def mean_ap_3(y_val,y_pred_score,k):
    y_pred = [np.argsort(c)[::-1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def check_accuracy(loader, model):
    
    model.eval()  # set model to evaluation mode
    
    meanAP = []
    meanAP3 = []
    with torch.no_grad():
        for i ,(_,x, y) in enumerate(loader):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            y_pred,_ = model(x,None)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
            if i > 100:
                break
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        

        if (m3 > QDtrain["mAP3"] + 1) and (m3 > 20):
            if os.path.exists(QDtrain["file"]):
                os.remove(QDtrain["file"])
            QDtrain["file"] = str("model" + QDtrain["prefix"] + str(m3))
            torch.save(model.state_dict(),QDtrain["file"])
            QDtrain["mAP3"] = m3
            QDtrain["mAP"] = m1
        print('Got correct (%.2f,%.2f)' % (m1,m3 ))
        
        
def train(model, optimizer, epochs,args):
    """
    Train a model on quickdrawing.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: A Python integer giving the number of epochs to train for
    - args: argumetns for learning rate
    
    Returns: Nothing, but prints model accuracies during training.
    """

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loader_val = QD.QDloadStrokeData(val = True)
    loader_val = DataLoader(dataset=loader_val, batch_size=512,shuffle=False)
    
    # We try to aggregate the several batches together 
    # so that we could have a big batchsize to fill in GPU.
    # real_batch size = aggregated_batches * batch_size
    aggregated_batches = 1
    
    for e in range(epochs):

        train_map1 = AverageMeter()
        train_map3 = AverageMeter()
        
        #Learning rate decay
        adjust_learning_rate(optimizer, e, args)
        
        # We split the whole train dataset into 100 segments.
        for i in range(20):
            t1 = time.time()
            total_loss = 0
            train_dataset = QD.QDloadStrokeData(no=i,val = False,transforms = trans)          
            train_loader = DataLoader(dataset=train_dataset, batch_size=512,shuffle=False)
            for t, (_, x, y) in enumerate(train_loader):
                model.train()  
                x = x.to(device=device, dtype=dtype)  
                y = y.to(device=device, dtype=torch.long)
                
                #add the center feature returned from resnet 
                scores,_ = model(x,None)
                #Caculate entropy loss
                entropy_loss = F.cross_entropy(scores, y)
                
                total_loss += entropy_loss
                #Calculate train accuracy
                y_pred = scores.data.cpu().numpy()
                y_val = y.reshape(-1,1)
                
                mAP = 100 * mean_ap_3(y_val,y_pred,1)
                mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
             
                #train_map1.update(mAP,x.size(0))
                #train_map3.update(mAP3,x.size(0))         
                                  
                if t % aggregated_batches == 0:                    
                    avg_loss = total_loss / aggregated_batches

                    optimizer.zero_grad()
                    avg_loss.backward()
                    optimizer.step()
                    total_loss = 0

                    t2 = time.time()

                    if t % (print_every) == 0:
                        print('Epoch %d,Iteration %d,loss = %.4f,time %.4f,train accuracy(%.2f,%.2f)' % 
                              (e,t, avg_loss.item(),mAP,mAP3))
                        check_accuracy(loader_val, model)
                        t1 = time.time()
                        

In [ ]:
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"rnn"}
#args = {"learning_rate" : 3e-4}
rnn_model = SketchRNN(3, 256, 40,dropout=0.1)
optimizer = optim.Adam(rnn_model.parameters(),lr = args['learning_rate'])
train(rnn_model,optimizer, epochs=5,args=args)


## Super Sketch Net

In [8]:
# Super Sketch Network links a RNN and CNN together with an attention layer in the last layer.
class SSN(nn.Module):
    
    def __init__(self, cnn_model_name,rnn_model_name, d_frozen = False,num_classes=40):
        super(SSN, self).__init__()
        
        self.cnn = RN.resnet50(num_classes=num_classes)
        self.rnn = SketchRNN(3, 256, num_classes,dropout=0.1)
        
        self.attention = nn.Parameter(torch.FloatTensor(num_classes, 1))
        torch.nn.init.xavier_normal_(self.attention)
        
        if os.path.exists(cnn_model_name):
            self.cnn.load_state_dict(torch.load(cnn_model_name, map_location=lambda storage, loc: storage))

        if os.path.exists(rnn_model_name):
            self.rnn.load_state_dict(torch.load(rnn_model_name, map_location=lambda storage, loc: storage))
            
        if d_frozen:
            for param in self.cnn.parameters():
                param.requires_grad = False
            for param in self.rnn.parameters():
                param.requires_grad = False
                
        
    def forward(self, images,strokes):
        cnn_output,cnn_f = self.cnn(images)
        rnn_output,rnn_f = self.rnn(strokes,None)
        
        #Attention Layer linking RNN and CNN together.
        output = torch.stack([cnn_output,rnn_output],dim = 1)
        
        #Get the center feature
        #print(cnn_f.shape,rnn_f.shape)
        ssn_feat = torch.cat((cnn_f,rnn_f),dim = 1)
        att_score = torch.matmul(output, self.attention).squeeze()
        att_score = F.softmax(att_score,dim = 1).view(output.size(0), output.size(1), 1)
        score = output * att_score

        score = torch.sum(score, dim=1)
        
        return score,ssn_feat

In [9]:
# The following codes comes from cs231N.
from average_precision import mapk
def mean_ap_torch(y_val,y_pred_score,k):
    y_pred = [torch.sort(c,descending=True)[::-1][1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def mean_ap_3(y_val,y_pred_score,k):
    y_pred = [np.argsort(c)[::-1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def check_accuracy(loader, model):
    
    model.eval()  # set model to evaluation mode
    
    meanAP = []
    meanAP3 = []
    with torch.no_grad():
        for i ,(img,stroke, y) in enumerate(loader):
            img = img.to(device=device, dtype=dtype)
            stroke = stroke.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            y_pred,_ = model(img,stroke)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
            if i > 100:
                break
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        

        if (m3 > QDtrain["mAP3"] + 0.1) and (m3 > 80):
            if os.path.exists(QDtrain["file"]):
                os.remove(QDtrain["file"])
            QDtrain["file"] = str("model" + QDtrain["prefix"] + str(m3))
            torch.save(model.state_dict(),QDtrain["file"])
            QDtrain["mAP3"] = m3
            QDtrain["mAP"] = m1
        print('Got correct (%.2f,%.2f)' % (m1,m3 ))
        
        
def train(model, optimizer, epochs,args):
    """
    Train a model on quickdrawing.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: A Python integer giving the number of epochs to train for
    - args: argumetns for learning rate
    
    Returns: Nothing, but prints model accuracies during training.
    """

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loader_val = QD.QDloadStrokeData(val = True)
    loader_val = DataLoader(dataset=loader_val, batch_size=256,shuffle=False)
    
    # We try to aggregate the several batches together 
    # so that we could have a big batchsize to fill in GPU.
    # real_batch size = aggregated_batches * batch_size
    aggregated_batches = 1
    
    for e in range(epochs):

        train_map1 = AverageMeter()
        train_map3 = AverageMeter()
        
        #Learning rate decay
        adjust_learning_rate(optimizer, e, args)
        
        # We split the whole train dataset into 100 segments.
        for i in range(20):
            t1 = time.time()
            total_loss = 0
            train_dataset = QD.QDloadStrokeData(no=i,val = False,transforms = trans)          
            train_loader = DataLoader(dataset=train_dataset, batch_size=256,shuffle=False)
            for t, (img, stroke, y) in enumerate(train_loader):
                model.train()  
                img = img.to(device=device, dtype=dtype)  
                stroke = stroke.to(device=device, dtype=dtype)
                y = y.to(device=device, dtype=torch.long)
                
                #add the center feature returned from resnet 
                scores,cf_pred = model(img,stroke)
                
                #Caculate entropy loss
                entropy_loss = F.cross_entropy(scores, y)

                #Caculate the center loss 
                #center_loss = F.mse_loss(cf_pred,cf_ssn[y])
                
                loss = entropy_loss #+ alpha * center_loss
                
                total_loss += loss
                                        
                if t % aggregated_batches == 0:                    
                    avg_loss = total_loss / aggregated_batches  
                    
                    optimizer.zero_grad()
                    avg_loss.backward()
                    optimizer.step()
                    total_loss = 0

                    t2 = time.time()

                    if t % (print_every) == 0:
                        
                        #Calculate train accuracy
                        y_pred = scores.data.cpu().numpy()
                        y_val = y.reshape(-1,1)

                        mAP = 100 * mean_ap_3(y_val,y_pred,1)
                        mAP3 = 100 * mean_ap_3(y_val,y_pred,3)

                        #train_map1.update(mAP,img.size(0))
                        #train_map3.update(mAP3,img.size(0)) 
                    
                        print('Epoch %d,Iteration %d,loss = %.4f,time %.4f,train accuracy(%.2f,%.2f)' % 
                              (e,t, avg_loss.item(),t2-t1,mAP,mAP3))
                        check_accuracy(loader_val, model)
                        t1 = time.time()
                        

In [ ]:
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"SSN"}
args["learning_rate"] = 3e-3
ssn_model = SSN("model","model")
optimizer = optim.Adam(ssn_model.parameters(),lr = args['learning_rate'])
train(ssn_model,optimizer, epochs=5,args=args)

No = 0 and total number of items 20480
No = 0 and total number of items 281600
Epoch 0,Iteration 0,loss = 3.6948,time 113.0431,train accuracy(2.34,5.08)
Got correct (2.50,4.58)
Epoch 0,Iteration 300,loss = 1.7464,time 111.3783,train accuracy(49.61,61.65)
Got correct (48.83,60.96)
Epoch 0,Iteration 600,loss = 1.1317,time 112.4751,train accuracy(67.58,76.30)
Got correct (57.78,68.75)
Epoch 0,Iteration 900,loss = 0.8846,time 112.3417,train accuracy(70.31,79.23)
Got correct (60.52,71.63)
No = 1 and total number of items 280576
Epoch 0,Iteration 0,loss = 0.9887,time 114.0976,train accuracy(67.58,77.41)
Got correct (68.87,78.24)
Epoch 0,Iteration 300,loss = 0.8659,time 112.0369,train accuracy(73.44,82.42)
Got correct (72.10,80.95)
Epoch 0,Iteration 600,loss = 0.7173,time 113.2345,train accuracy(76.17,84.44)
Got correct (73.02,81.77)
Epoch 0,Iteration 900,loss = 0.8096,time 112.6432,train accuracy(71.88,80.92)
Got correct (73.96,82.35)
No = 2 and total number of items 280576
Epoch 0,Iteration

No = 18 and total number of items 281600
Epoch 0,Iteration 0,loss = 0.4634,time 115.0079,train accuracy(83.98,90.36)
Got correct (83.25,89.34)
Epoch 0,Iteration 300,loss = 0.4906,time 113.4102,train accuracy(83.98,89.78)
Got correct (83.40,89.56)
Epoch 0,Iteration 600,loss = 0.5399,time 114.0378,train accuracy(83.20,88.67)
Got correct (83.39,89.51)
Epoch 0,Iteration 900,loss = 0.4967,time 113.8596,train accuracy(82.81,89.13)
Got correct (83.10,89.36)
No = 19 and total number of items 281600
Epoch 0,Iteration 0,loss = 0.5511,time 114.5427,train accuracy(83.20,88.93)
Got correct (83.33,89.44)
Epoch 0,Iteration 300,loss = 0.5043,time 113.2953,train accuracy(80.08,87.43)
Got correct (83.24,89.43)
Epoch 0,Iteration 600,loss = 0.4769,time 113.8652,train accuracy(86.33,91.34)
Got correct (83.19,89.44)
Epoch 0,Iteration 900,loss = 0.4680,time 114.0670,train accuracy(82.42,89.13)
Got correct (83.20,89.39)
No = 0 and total number of items 281600
Epoch 1,Iteration 0,loss = 0.4642,time 115.2399,tr

# Run test samples
* Run the whole test daset to get the accuracy.

In [13]:
def evaluate_test(test_data, model,args):
    model.eval()  
    with torch.no_grad():
        meanAP = []
        meanAP3 = []
        center_feature = []
        y_eval_list = []
        
        for i ,(img,stroke,y) in enumerate(test_data):
            img = img.to(device=device, dtype=dtype)
            stroke = stroke.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            
            if args['model']=='ssn':
                y_pred,feature_pred = model(img,stroke)
                feature_pred = feature_pred.data.cpu().numpy()
                center_feature.extend(feature_pred)
                
            if args['model']=='cnn':
                y_pred,feature_pred = model(img)
                feature_pred = feature_pred.data.cpu().numpy()
                center_feature.extend(feature_pred)
                #print(len(center_feature),center_feature[0].shape)
                
            if args['model']=='rnn':
                y_pred,_ = model(stroke,None)
                
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            y_eval_list.append((y_val.data.cpu().numpy(),np.argmax(y_pred,axis=1)))
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
    
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        
        return (m1,m3,center_feature,y_eval_list)

In [14]:
model = ssn_model

In [15]:
args['model'] = 'ssn'

In [16]:
model = model.to(device=device)
checkpoint = torch.load("ssn_model_loss_center_89.50")
model.load_state_dict(checkpoint)

class_name = QD.qd_names

class_accuracy = []
center_feature = []
y_list = []

val_accuracy_map1 = AverageMeter()
val_accuracy_map3 = AverageMeter()

for i,name in enumerate(tqdm(class_name)):
    t1 = time.time()
    batch_size = 128
    data_file = os.path.join("./train",name+".csv")
    test_data = QD.QDloadStrokeData(data_file = data_file)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, num_workers=2, shuffle=False)

    m1,m3,feature_list,y_eval_list = evaluate_test(test_loader,model,args)
    
    if args['model'] != 'rnn':
        feature_list = np.array(feature_list)
        print(feature_list.shape)

        #feature_list is N * 2048
        center_feature.append(np.mean(feature_list,axis = 0))
        
    class_accuracy.append((name,str(m1),str(m3)))
    y_list.extend(y_eval_list)
    
    val_accuracy_map1.update(m1,len(y_eval_list))
    val_accuracy_map3.update(m3,len(y_eval_list))
    
    print(name,str(m1),str(m3))
    t2 = time.time()
    print("Time:",t2-t1)
    
with open('class_accuracy.csv',"w+") as fp:
    writer = csv.writer(fp, delimiter=',')
    writer.writerow(["class", "meanAP1", "meanAP3"])  # write header
    writer.writerows(class_accuracy)

print("The average accuracy %.4f %.4f" % (val_accuracy_map1.avg,val_accuracy_map3.avg))
np.save("y_test_list",y_list)

if args['model'] != 'rnn':
    np.save("center_feature_" + args['model'],center_feature)
    

  0%|          | 0/40 [00:00<?, ?it/s]

No = 0 and total number of items 116623


  2%|▎         | 1/40 [01:59<1:17:35, 119.38s/it]

(116623, 2088)
cup 51.468555 71.587944
Time: 119.38303399085999
No = 0 and total number of items 108633


  5%|▌         | 2/40 [03:51<1:14:14, 117.22s/it]

...

(125651, 2088)
donut 97.640335 98.207306
Time: 137.56318950653076
No = 0 and total number of items 121337


 98%|█████████▊| 39/40 [1:39:14<02:23, 143.77s/it]

(121337, 2088)
hourglass 96.160995 96.85629
Time: 120.25450611114502
No = 0 and total number of items 107776


100%|██████████| 40/40 [1:41:27<00:00, 140.51s/it]

(107776, 2088)
bee 94.430115 96.20262
Time: 132.92423796653748
The average accuracy 84.6354 90.3293


* Run the test dataset to evaluate the performance of SSN.

In [17]:
args['model'] = 'ssn'
model = ssn_model
#model = cnn_model.to(device=device)
#checkpoint = torch.load("modelresnet5091.17106")
#model.load_state_dict(checkpoint)

def run_testdataset(args,model,file_name = "test_dataset.csv"):
    batch_size = 128
    data_file = os.path.join("./test",file_name)
    test_data = QD.QDloadStrokeData(data_file = data_file)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, num_workers=2, shuffle=False)

    m1,m3,feature_list,y_eval_list = evaluate_test(test_loader,model,args)
    print(" Got the correct %.4f %.4f classification" % (m1,m3))
    return y_eval_list

y_eval_list = run_testdataset(args,model) 
    

No = 0 and total number of items 20480
 Got the correct 84.3555 90.0716 classification


In [19]:
np.save("center_feature_" + args['model'],center_feature)

In [40]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

for i,item in enumerate(y_eval_list):
    y_list = zip(item)
    print(item)
df_cm = pd.DataFrame(y_pred, index = [name for name in QD.qd_names],
                  columns = [name for name in QD.qd_names])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

(array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
   

ValueError: Shape of passed values is (40, 128), indices imply (40, 40)

1) 96*96, resnet50, epoch 10, GSD. 75.2%
2) 96*96, resnet50, epoch 10, GSD  79.9% model_parametertensor(91.6667)
3) 96*96, resnet50, epoch 10, Adam model_resnet_92.317_90


* convert simplified test dataset to pixel drawsing

Check the accuracy based on the class.

Create test_simplified.
load compressed(96*96) 1.968482494354248
load non compressed(96*96)  0.31299567222595215
load non compressed(28*28) 0.04715704917907715


In [11]:
torch.save(ssn_model.state_dict(),"ssn_model_loss_center_e1_89.50")

In [18]:
if args['model'] == 'cnn':
    np.save("center_feature",center_feature)

In [ ]:
torch.cuda.empty_cache()